In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from helper_functions import *

# Prep data

## Import

In [ ]:
#change the homepath to the directory "EEDIPy" has been cloned to
homepath = 'F:/projects/EEDIPy/'
# inpt = pd.read_excel(homepath + 'inputs_new.xlsx')

### Verification

In [ ]:
#un-comment verifications you want to run
#mepc 79
# inpt = pd.read_excel(homepath + 'verifications/mepc_79_1.xlsx')
# inpt = pd.read_excel(homepath + 'verifications/mepc_79_2.xlsx')
# inpt = pd.read_excel(homepath + 'verifications/mepc_79_3.xlsx')
# inpt = pd.read_excel(homepath + 'verifications/mepc_79_4.xlsx')
# inpt = pd.read_excel(homepath + 'verifications/mepc_79_5.xlsx')

#publication 103P
# inpt = pd.read_excel(homepath + 'verifications/pr_38_rev4_1.xlsx')
# inpt = pd.read_excel(homepath + 'verifications/pr_38_rev4_2.xlsx')
# inpt = pd.read_excel(homepath + 'verifications/pr_38_rev4_3.xlsx')
# inpt = pd.read_excel(homepath + 'verifications/pr_38_rev4_4.xlsx')
# inpt = pd.read_excel(homepath + 'verifications/pr_38_rev4_5.xlsx')
# inpt = pd.read_excel(homepath + 'verifications/pr_38_rev4_6.xlsx')
# inpt = pd.read_excel(homepath + 'verifications/pr_38_rev4_app_6_1.xlsx')
# inpt = pd.read_excel(homepath + 'verifications/pr_38_rev4_app_6_2.xlsx')
# inpt = pd.read_excel(homepath + 'verifications/pr_38_rev4_app_6_3.xlsx')

#examples
# inpt = pd.read_excel(homepath + 'examples/mepc_79_1_innovative.xlsx')


## Variables

In [ ]:
#main variables
df_inpt = load_variables(inpt)

#cf dictionary
cf_dict = load_cf_dict(inpt)

#me data
df_me = load_me_data(inpt, df_inpt)
mcr_me = df_me['mcr'].sum()
p_me = df_me['p_me'].sum()

#ae data
df_ae = load_ae_data(inpt)


# Calculation

## Derived inputs

In [ ]:
#sfc
df_me, df_ae = calculate_sfc(df_me, df_ae, cf_dict)

#cf
df_me, df_ae = calculate_cf(df_me, df_ae, cf_dict)

# if df_inpt['ship_type'].iloc[0] == 'cruise_ship':
#     df_inpt['dwt'] = df_inpt['gt'].iloc[0].copy()
    
capacity = capacity_calc(dwt = df_inpt['dwt'].iloc[0],
                         ship_type = df_inpt['ship_type'].iloc[0])

disp_t = df_inpt['disp_m3'].iloc[0] * 1.025


## Pae

In [ ]:
sfc_me_df = df_me.loc[df_me['engine_type']=='dual_fuel',['sfc_pilot_fuel','sfc_gas_fuel']].sum().sum()

if df_inpt['hload'].iloc[0] == 0:
    p_ae = p_ae_iterative_calc(ship_type = df_inpt['ship_type'].iloc[0],
                               mcr_me = mcr_me,
                               me_type = df_inpt['propulsion_type'].iloc[0],
                               p_sm_rated = df_inpt['p_sm_rated'].iloc[0],
                               mpp = df_inpt['mpp'].iloc[0],
                               p_pto_rated = df_inpt['p_pto_rated'].iloc[0],
                               cube = df_inpt['cube'].iloc[0],
                               me_engine_stroke = df_inpt['me_engine_stroke'].iloc[0],
                               sfc_me_gas_mode = sfc_me_df,
                               electrical_eff = df_inpt['electrical_eff'].iloc[0],  
                               gen_efficiency = df_inpt['gen_efficiency'].iloc[0],
                               pti_eff = df_inpt['pti_eff'].iloc[0],
                               bor = df_inpt['bor'].iloc[0],
                               cop_cooling = df_inpt['cop_cooling'].iloc[0],
                               r_reliq = df_inpt['r_reliq'].iloc[0],
                               cop_comp = df_inpt['cop_comp'].iloc[0],                               
                               add_load = df_inpt['p_ae_eff_al'].iloc[0],
                               )    
    
elif df_inpt['hload'].iloc[0] > 0:
    p_ae = df_inpt['hload'].iloc[0] / df_inpt['gen_efficiency'].iloc[0]

## PTO

In [ ]:
#pto calculation power
p_pto = p_pto_calc(p_pto_rated=df_inpt['p_pto_rated'].iloc[0],
                   me_type=df_inpt['propulsion_type'].iloc[0])

p_pto_remove_me, p_ae_calc = pto_pae_ratio(df_inpt, df_me, df_ae, p_ae, p_pto)
    
df_me['pto_remove'] = p_pto_remove_me
df_ae['p_ae_calc'] = p_ae_calc

#if using engine limitation, pto calculation option 2 is used
if sum(df_me['limited_power']) > 0:
    df_me['p_me_calc'] = df_me['p_me']
else:
    df_me['p_me_calc'] = df_me['p_me'] - df_me['pto_remove']

if df_inpt['mpp'].iloc[0] == 0:
    p_me = df_me['p_me_calc'].sum()
elif (df_inpt['mpp'].iloc[0] > 0) and (df_inpt['ship_type'].iloc[0] == 'cruise_ship'):
    p_me = 0


## PTI

In [ ]:
if df_inpt['mpp'].iloc[0] == 0:
    p_pti = shaft_motor_power(p_sm_rated = df_inpt['p_sm_rated'].iloc[0],
                            me_type = df_inpt['propulsion_type'].iloc[0],
                            mpp = df_inpt['mpp'].iloc[0],
                            gen_efficiency = df_inpt['gen_efficiency'].iloc[0],
                            pti_eff = df_inpt['pti_eff'].iloc[0])[0]

    p_pti_shaft = shaft_motor_power(p_sm_rated = df_inpt['p_sm_rated'].iloc[0],
                                    me_type = df_inpt['propulsion_type'].iloc[0],
                                    mpp = df_inpt['mpp'].iloc[0],
                                    gen_efficiency = df_inpt['gen_efficiency'].iloc[0],
                                    pti_eff = df_inpt['pti_eff'].iloc[0])[1]
elif (df_inpt['mpp'].iloc[0] > 0) & (df_inpt['ship_type'].iloc[0] == 'cruise_ship'):
    p_pti = shaft_motor_power(p_sm_rated = df_inpt['mpp'].iloc[0],
                              me_type = df_inpt['propulsion_type'].iloc[0],
                              mpp = df_inpt['mpp'].iloc[0],
                              gen_efficiency = df_inpt['gen_efficiency'].iloc[0],
                             pti_eff = df_inpt['pti_eff'].iloc[0])[0]

    p_pti_shaft = shaft_motor_power(p_sm_rated = df_inpt['mpp'].iloc[0],
                                    me_type = df_inpt['propulsion_type'].iloc[0],
                                    mpp = df_inpt['mpp'].iloc[0],
                                    gen_efficiency = df_inpt['gen_efficiency'].iloc[0],
                                    pti_eff = df_inpt['pti_eff'].iloc[0])[1]
else:
    p_pti = 0
    p_pti_shaft = 0

p_me_deduct = p_me + p_pti_shaft - p_pto_remove_me

v_ref = update_vref(df_inpt, p_me_deduct)


## Fuel ratio

In [ ]:
fd_gas, df_me, df_ae = fuel_ratio_calc(df_inpt, df_me, df_ae)

## Gather terms

In [ ]:
#me and pto terms
cf_sfc_me, me_term, pto_term, df_me = me_term_calc(df_me)
if np.isnan(cf_sfc_me):
    cf_sfc_me = np.nan_to_num(cf_sfc_me)
#ae term
cf_sfc_ae, ae_term, df_ae = ae_term_calc(df_ae, cf_sfc_me)

## Innovative

In [ ]:
c_1_val = cat_c1(w_e = df_inpt['w_e'].iloc[0],
                 eta_g = df_inpt['eta_g'].iloc[0],
                 p_ae_eff_loss = df_inpt['p_ae_eff_loss'].iloc[0])

c_2_val = cat_c2(f_temp = df_inpt['f_temp'].iloc[0],
                 p_max = df_inpt['p_max'].iloc[0],
                 etad_gen = df_inpt['etad_gen'].iloc[0],
                 n = df_inpt['n'].iloc[0],
                 f_rad = 0.2,
                 l_others = 10)

p_eff = cat_b1_short(p_p_eff_al = df_inpt['p_p_eff_al'].iloc[0],
                     p_ae_eff_al = df_inpt['p_ae_eff_al'].iloc[0],
                     p_me = p_me,
                     p_pti_shaft = p_pti_shaft,
                     cf_sfc_me = cf_sfc_me, cf_sfc_ae = cf_sfc_ae)[0]

cf_sfc_me_pti = cat_b1_short(p_p_eff_al = df_inpt['p_p_eff_al'].iloc[0],
                             p_ae_eff_al = df_inpt['p_ae_eff_al'].iloc[0],
                             p_me = p_me, p_pti_shaft = p_pti_shaft,
                             cf_sfc_me = cf_sfc_me, cf_sfc_ae = cf_sfc_ae)[1]


b1_term = p_eff * cf_sfc_me_pti

## Correction factors

In [ ]:
"""
fj
    for ice-classed ships
    shuttle tankers with propulsion redundancy
    ro-ro cargo and ro-ro passenger ships
    general cargo ships
fi
    ice class
    structural enhancement - use not from standard
    common structural rules
fc
    chemical tankers as defined in regulation 1.16.1 of MARPOL Annex II
    gas carriers having direct diesel driven propulsion system constructed or adapted and used
        for the carriage in bulk of liquefied natural gas
    For ro-ro passenger ships having a DWT/GT-ratio of less than 0.25
    For bulk carriers having R of less than 0.55
fl
    general cargo ships equipped with cranes and other cargo-related gear
fw
    Factor for speed reduction at sea
fm
    For ice-classed ships having IA Super or IA
"""

### fj

In [ ]:
fj_term = fj(ship_type=df_inpt['ship_type'].iloc[0],
             ice_class=df_inpt['ice_class'].iloc[0],
             mcr=mcr_me,
             dwt=df_inpt['dwt'].iloc[0],
             propulsion_redundancy=df_inpt['propulsion_redundancy'].iloc[0],
             l=df_inpt['lpp'].iloc[0],
             b=df_inpt['b'].iloc[0],
             d=df_inpt['ds'].iloc[0],
             disp_m3=df_inpt['disp_m3'].iloc[0],
             v_ref=v_ref,
             g=9.81)

### fi

In [ ]:
fi_term = fi(ship_type=df_inpt['ship_type'].iloc[0],
             csr=df_inpt['csr'].iloc[0],
             ice_class=df_inpt['ice_class'].iloc[0],
             dwt=df_inpt['dwt'].iloc[0],
             l=df_inpt['lpp'].iloc[0],
             b=df_inpt['b'].iloc[0],
             d=df_inpt['ds'].iloc[0],
             disp_m3=df_inpt['disp_m3'].iloc[0],
             disp_t=disp_t,
             lwt_ref=df_inpt['lwt_ref'].iloc[0],
             lwt_enhance=df_inpt['lwt_enhance'].iloc[0],
             lwt_csr=df_inpt['lwt_csr'].iloc[0],
             dwt_csr=df_inpt['dwt_csr'].iloc[0])

### fc

In [ ]:
fc_term = fc(ship_type=df_inpt['ship_type'].iloc[0],
             dwt=capacity,
             cube=df_inpt['cube'].iloc[0],
             diesel_direct_drive=df_inpt['diesel_direct_drive'].iloc[0],
             marpol_annex=df_inpt['marpol_annex'].iloc[0],
             gt=df_inpt['gt'].iloc[0])

### fl

In [ ]:
fl_term = fl(ship_type=df_inpt['ship_type'].iloc[0],
             dwt_ref=df_inpt['dwt'].iloc[0],
             number_of_cranes=df_inpt['number_of_cranes'].iloc[0],
             swl_crane=df_inpt['swl_crane'].iloc[0],
             reach_crane=df_inpt['reach_crane'].iloc[0],
             side_loader_weight=df_inpt['side_loader_weight'].iloc[0],
             roro_weight=df_inpt['roro_weight'].iloc[0])

### fw

In [ ]:
fw_term = 1

### fm

In [ ]:
fm_term = fm(ice_class=df_inpt['ice_class'].iloc[0])

## PTI term

In [ ]:
if df_ae['ae_term'].sum() == 0:
    pti_term = 0
    pti_and_c_term = 0
else:
    pti_term = (fj_term * p_pti)* cf_sfc_ae
    pti_and_c_term = ((fj_term * p_pti) - (c_1_val + c_2_val))* cf_sfc_ae

# Final EEDI

## Check values

In [ ]:
print('P me term = {:.0f}'.format(p_me))
print('CFme x SFC me = {:.0f}'.format(cf_sfc_me))
print('P ae term = {:.0f}'.format(p_ae))
print('CFae x SFC ae = {:.0f}'.format(cf_sfc_ae))
print('pto power off shaft = {:.0f}'.format(p_pto_remove_me * len(df_me)))
print('pti power on shaft = {:.0f}'.format(p_pti_shaft))

In [ ]:
print('gas ratio = {:.4f}'.format(fd_gas))

In [ ]:
print('fj term = {:.2f}'.format(fj_term))
print('me term = {:.0f}'.format(me_term))
print('pto term = {:.0f}'.format(pto_term))
print('ae term = {:.0f}'.format(ae_term))
print('pti term = {:.0f}'.format(pti_term))
print('fi term = {:.2f}'.format(fi_term))
print('fc term = {:.2f}'.format(fc_term))
print('fl term = {:.2f}'.format(fl_term))
print('capacity = {:.0f}'.format(capacity))
print('fw term = {:.2f}'.format(fw_term))
print('v ref = {:.2f}'.format(v_ref))
print('fm term = {:.2f}'.format(fm_term))

## EEDI

In [ ]:
eedi_no_tech = ((fj_term * me_term + pto_term + ae_term + pti_term)
                / (fi_term * fc_term * fl_term * capacity * fw_term * v_ref * fm_term))

eedi_with_tech = ((fj_term * me_term + pto_term + ae_term + pti_and_c_term - b1_term)
                  / (fi_term * fc_term * fl_term * capacity * fw_term * v_ref * fm_term))

print('EEDI (no energy saving tech) = {:.2f}'.format(eedi_no_tech))
print('EEDI (with energy saving tech) = {:.2f}'.format(eedi_with_tech))

# Plot

## EEDI Reference Line

In [ ]:
#import plotting references
ref_eq_df = pd.read_csv(homepath + 'resources/plotting_curves.csv')
df_reduct = pd.read_csv(homepath + 'resources/reduction_table.csv')
gt = 120000 #only required for roro_cargo_vehicle ships
dwt = capacity.copy()

In [ ]:
#fill in missing data in ref_eq_df based on ship parameters
if df_inpt['ship_type'].iloc[0] == 'roro_cargo_vehicle':
    ref_eq_df = roro_plot_calc(ref_eq_df, dwt, gt) 
     
ref_eq_df['b'] = dwt
dwt_lim = df_reduct[df_reduct['ship_type']==df_inpt['ship_type'].iloc[0]]

phase_1_frac, phase_2_frac, phase_3_frac = phase_frac_calc(dwt, dwt_lim)

## Output

In [ ]:
#create reference lines for plotting
ref_line_df = pd.DataFrame({'dwt':np.arange(100,dwt*2,100)})
ref_line_df['phase_0'] = ref_eq_df['a'][ref_eq_df['ship_type']==df_inpt['ship_type'].iloc[0]].item() \
    * ref_line_df['dwt'] \
    ** (-1* ref_eq_df['c'][ref_eq_df['ship_type']==df_inpt['ship_type'].iloc[0]].item())
ref_line_df['phase_1'] = (1 - (phase_1_frac) / 100) * ref_line_df['phase_0'] 
ref_line_df['phase_2'] = (1 - (phase_2_frac) / 100) * ref_line_df['phase_0'] 
ref_line_df['phase_3'] = (1 - (phase_3_frac) / 100) * ref_line_df['phase_0'] 


In [ ]:
#plot
plt.figure(figsize=(12,8))
plt.scatter(dwt, eedi_no_tech, label='eedi without energy saving tech = {:.2f}'.format(eedi_no_tech), s=100)
plt.scatter(dwt, eedi_with_tech, label='eedi with energy saving tech = {:.2f}'.format(eedi_with_tech), s=100)

plt.plot(ref_line_df['dwt'], ref_line_df['phase_0'],
            label = 'phase 0 reference line')
plt.plot(ref_line_df['dwt'], ref_line_df['phase_1'],
            label = 'phase 1 reference line')
plt.plot(ref_line_df['dwt'], ref_line_df['phase_2'],
            label = 'phase 2 reference line')
plt.plot(ref_line_df['dwt'], ref_line_df['phase_3'],
            label = 'phase 3 reference line')

plt.xlim(0, dwt*1.8)
plt.ylim(0,5 * round(eedi_no_tech / 5) * 2)

if df_inpt['ship_type'].iloc[0] == 'cruise_ship':
    plt.xlabel('Capacity [GT]')
else:
    plt.xlabel('Capacity [DWT]')
plt.ylabel('EEDI')
plt.title('Calculated EEDI')

plt.legend()
plt.grid()